In [1]:
import numpy as np  
import pandas as pd  
#importing modules, to see graphical analysis see the earliest .ipynb

In [2]:
df=pd.read_csv('train.csv')
#reading csv file

In [3]:
df.head()
#head of data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df=df.drop(['Cabin','Ticket','Name','PassengerId'],axis=1)
#dropping some not important columns

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
df=df.dropna()

In [7]:
sex=pd.get_dummies(df['Sex'],drop_first=True)
embark=pd.get_dummies(df['Embarked'],drop_first=True)
df=pd.concat([df,sex,embark],axis=1)
df=df.drop(['Sex','Embarked'],axis=1)
#making data numeric

In [8]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.7*len(df)), int(.8*len(df))])
#splitting data

In [9]:
X_train=train.drop('Survived',axis=1)
X_validation=validate.drop('Survived',axis=1)
X_test=test.drop('Survived',axis=1)
y_train=train[['Survived']]
y_validation=validate[['Survived']]
y_test=test[['Survived']]
#defining data

In [10]:
train.shape

(498, 9)

In [11]:
X_train.insert(0,'bias',value=1)
X_validation.insert(0,'bias',value=1)
X_test.insert(0,'bias',value=1)
#inserting bias

In [12]:
X_train.head()

,bias,Pclass,Age,SibSp,Parch,Fare,male,Q,S
641,1,1,24.0,0,0,69.3000,0,0,0
496,1,1,54.0,1,0,78.2667,0,0,0
262,1,1,52.0,1,1,79.6500,1,0,1
311,1,1,18.0,2,2,262.3750,0,0,0
551,1,2,27.0,0,0,26.0000,1,0,1


In [13]:
X_train.shape

(498, 9)

In [14]:
y_train.shape

(498, 1)

In [15]:
theta=np.zeros(X_train.shape[1]).reshape(1,X_train.shape[1])
#weight matrix

In [16]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))
#sigmoid function 

In [17]:
def cost(x,y,theta):
    m=len(x)
    z1=np.dot(y.T,np.log(sigmoid(np.dot(x,theta.T))))
    z2=np.dot((1-y).T,np.log(1-sigmoid(np.dot(x,theta.T))))
    z1=(z1+z2)/(-m)
    lam=0.0001
    total=np.sum((lam/(2*m))*(np.square(theta[:,1:])))
    total=total+z1
    return total
#cost function

In [18]:
cost(X_train,y_train,theta)
#checking cost

array([[0.69314718]])

In [19]:
def gradient_descent(x,y,theta):
    m=len(x)
    alpha=0.0001
    lam=0.0001
    y=np.matrix(y)
    costi=np.zeros(1000)
    for i in range(1000):
        z1=sigmoid(np.dot(x,theta.T))
        z1=z1-y
        z1=np.dot(z1.T,x)
        z1=z1/m
        z1[:,1:]=z1[:,1:]+(lam/m)*(theta[:,1:])
        z1=z1*alpha
        theta=theta-z1
        costi[i]=cost(x,y,theta)
    return theta,costi
    #gradient descent

In [20]:
f,g=gradient_descent(X_train,y_train,theta)

In [21]:
f
#modified weight

matrix([[-0.00200914, -0.01458571, -0.02442743, -0.00711265, -0.00057247,
          0.01142173, -0.01273935, -0.00093325, -0.00340425]])

In [22]:
g[99],g[499],g[999]

(0.6412021700763748, 0.6364239184971636, 0.6345761479377185)

In [23]:
pre=np.dot(X_validation,f.T)
#predictions for validation set

In [24]:
pre=pd.DataFrame(pre)

In [25]:
pre=sigmoid(pre)
#making predictions positive (0-1 range)

In [26]:
for i in range(pre.shape[0]):
    if pre.iloc[i,0]<=0.5:
        pre.iloc[i,0]=0
    if pre.iloc[i,0]>0.5:
        pre.iloc[i,0]=1
#defining values

In [27]:
sd=0
for i in range(pre.shape[0]):
    if pre.iloc[i,0]==y_validation.iloc[i,0]:
        sd=sd+1
print(sd/len(pre))
#accuracy

0.6901408450704225


In [28]:
#note that this accuracy can further increase if we increase iteration which again has a system limit

In [29]:
prediction=np.dot(X_test,f.T)

In [30]:
prediction=pd.DataFrame(prediction)

In [31]:
prediction=sigmoid(prediction)

In [32]:
for i in range(prediction.shape[0]):
    if prediction.iloc[i,0]<=0.5:
        prediction.iloc[i,0]=0
    if prediction.iloc[i,0]>0.5:
        prediction.iloc[i,0]=1

In [33]:
sd=0
for i in range(prediction.shape[0]):
    if prediction.iloc[i,0]==y_test.iloc[i,0]:
        sd=sd+1
print(sd/len(prediction))
#accuracy on test set

0.7062937062937062
